In [1]:
import pandas as pd
import numpy as np
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import nltk
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from imblearn.over_sampling import RandomOverSampler
import time
from imblearn.over_sampling import SMOTE
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
df = pd.read_csv("clean_dataset.csv")

df

,at,userName,score,content,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir,polarity_score,polarity
0,2025-03-29 16:56:30,Pengguna Google,2,mengapa sangat sulit mengganti KTP padahal sem...,mengapa sangat sulit mengganti KTP padahal sem...,mengapa sangat sulit mengganti ktp padahal sem...,mengapa sangat sulit mengganti ktp padahal sem...,"['mengapa', 'sangat', 'sulit', 'mengganti', 'k...","['sulit', 'mengganti', 'ktp', 'persyaratan', '...",sulit mengganti ktp persyaratan terpenuhi vide...,2,positive
1,2025-03-27 18:49:41,Pengguna Google,1,Emang aplikasi BUSUK.Saya transaksi sudah terj...,Emang aplikasi BUSUKSaya transaksi sudah terja...,emang aplikasi busuksaya transaksi sudah terja...,emang aplikasi busuksaya transaksi sudah terja...,"['emang', 'aplikasi', 'busuksaya', 'transaksi'...","['emang', 'aplikasi', 'busuksaya', 'transaksi'...",emang aplikasi busuksaya transaksi berhasil bu...,8,positive
2,2025-04-08 10:18:29,Pengguna Google,2,kurangin bintang nya ga bisa di buka kalo jari...,kurangin bintang nya ga bisa di buka kalo jari...,kurangin bintang nya ga bisa di buka kalo jari...,kurangin bintang nya ga bisa di buka kalo jari...,"['kurangin', 'bintang', 'nya', 'ga', 'bisa', '...","['kurangin', 'bintang', 'buka', 'kalo', 'jarin...",kurangin bintang buka kalo jaringan data mah p...,-9,negative
3,2025-04-08 23:00:03,Pengguna Google,3,"Dulu sbnrnya ini apk bgs, tapi lama lama error...",Dulu sbnrnya ini apk bgs tapi lama lama error ...,dulu sbnrnya ini apk bgs tapi lama lama error ...,dulu sbnrnya ini apk bagus tapi lama lama erro...,"['dulu', 'sbnrnya', 'ini', 'apk', 'bagus', 'ta...","['sbnrnya', 'apk', 'bagus', 'error', 'isi', 's...",sbnrnya apk bagus error isi saldo error tf err...,-17,negative
4,2025-04-03 05:24:42,Pengguna Google,1,"Akun tiba"" terhubung ke UC Drive Premium, Sald...",Akun tiba terhubung ke UC Drive Premium Saldo ...,akun tiba terhubung ke uc drive premium saldo ...,akun tiba terhubung ke uc drive premium saldo ...,"['akun', 'tiba', 'terhubung', 'ke', 'uc', 'dri...","['akun', 'terhubung', 'uc', 'drive', 'premium'...",akun terhubung uc drive premium saldo terpoton...,-12,negative
...,...,...,...,...,...,...,...,...,...,...,...,...
49995,2025-01-14 01:08:50,NOVI ARDANA,5,aplikasi dana sangat membantu untuk tf di mana...,aplikasi dana sangat membantu untuk tf di mana...,aplikasi dana sangat membantu untuk tf di mana...,aplikasi Dana sangat membantu untuk tf di mana...,"['aplikasi', 'Dana', 'sangat', 'membantu', 'un...","['aplikasi', 'Dana', 'membantu', 'tf', 'lpun']",aplikasi Dana membantu tf lpun,-4,negative
49996,2019-11-15 20:12:48,Pengguna Google,1,"Saya baru pake aplikasi ini, karna ada kepenti...",Saya baru pake aplikasi ini karna ada kepentin...,saya baru pake aplikasi ini karna ada kepentin...,saya baru pake aplikasi ini karna ada kepentin...,"['saya', 'baru', 'pake', 'aplikasi', 'ini', 'k...","['pake', 'aplikasi', 'karna', 'kepentingan', '...",pake aplikasi karna kepentingan kerjaan downlo...,-4,negative
49997,2023-04-10 15:29:46,Raja Malam,1,Ini gimana si dulu mudah mudah aja masuk akun ...,Ini gimana si dulu mudah mudah aja masuk akun ...,ini gimana si dulu mudah mudah aja masuk akun ...,ini gimana si dulu mudah mudah aja masuk akun ...,"['ini', 'gimana', 'si', 'dulu', 'mudah', 'muda...","['gimana', 'si', 'mudah', 'mudah', 'aja', 'mas...",gimana si mudah mudah aja masuk akun Dana suli...,4,positive
49998,2021-05-07 21:27:09,Ahmad Saipullah,1,Lama bener ni lelet. Udh di wa antrian selalu ...,Lama bener ni lelet Udh di wa antrian selalu p...,lama bener ni lelet udh di wa antrian selalu p...,lama bener ni lambat udh di wa antrian selalu ...,"['lama', 'bener', 'ni', 'lambat', 'udh', 'di',...","['bener', 'ni', 'lambat', 'udh', 'wa', 'antria...",bener ni lambat udh wa antrian penuh ditanggap...,-3,negative


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   at                    50000 non-null  object
 1   userName              50000 non-null  object
 2   score                 50000 non-null  int64 
 3   content               50000 non-null  object
 4   text_clean            50000 non-null  object
 5   text_casefoldingText  50000 non-null  object
 6   text_slangwords       50000 non-null  object
 7   text_tokenizingText   50000 non-null  object
 8   text_stopword         50000 non-null  object
 9   text_akhir            49993 non-null  object
 10  polarity_score        50000 non-null  int64 
 11  polarity              50000 non-null  object
dtypes: int64(2), object(10)
memory usage: 4.6+ MB


In [4]:
df.isnull().sum()

,0
at,0
userName,0
score,0
content,0
text_clean,0
text_casefoldingText,0
text_slangwords,0
text_tokenizingText,0
text_stopword,0
text_akhir,7


In [5]:
df = df.dropna(subset=['text_akhir'])

In [6]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
X = df['text_akhir']
y = df['polarity']

le = LabelEncoder()
y_encoded = le.fit_transform(y)

#Pelatihan: SVM,    Ekstraksi Fitur: TF-IDF,    Pembagian Data: 80/20

In [7]:
tfidf = TfidfVectorizer(max_features=5000)
X_tfidf = tfidf.fit_transform(X)

print("TF-IDF shape:", X_tfidf.shape)

X_train2, X_test2, y_train2, y_test2 = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

label_encoder = LabelEncoder()
label_encoder.fit(pd.concat([y_train2, y_test2]))
y_train_encoded = label_encoder.transform(y_train2)
y_test_encoded = label_encoder.transform(y_test2)

sm = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = sm.fit_resample(X_train2, y_train_encoded)


TF-IDF shape: (49993, 5000)


In [8]:
svm_model = SVC(kernel='linear')

start_time = time.time()
svm_model.fit(X_train_resampled, y_train_resampled)
print("Training selesai dalam", round(time.time() - start_time, 2), "detik")

y_pred_train = svm_model.predict(X_train2)
y_pred_test = svm_model.predict(X_test2)
y_pred_train_original = label_encoder.inverse_transform(y_pred_train)
y_pred_test_original = label_encoder.inverse_transform(y_pred_test)

accuracy_train = accuracy_score(y_train2, y_pred_train_original)
accuracy_test = accuracy_score(y_test2, y_pred_test_original)

print('SVM - akurasi data train:', accuracy_train)
print('SVM - akurasi data test:', accuracy_test)


Training selesai dalam 619.63 detik
SVM - akurasi data train: 0.9349402410361555
SVM - akurasi data test: 0.8807880788078808


#Pelatihan: RF,    Ekstraksi Fitur: TF-IDF,    Pembagian Data: 80/20  

In [9]:
tfidf = TfidfVectorizer(max_features=5000)
X_tfidf = tfidf.fit_transform(X)

X_train3, X_test3, y_train3, y_test3 = train_test_split(X_tfidf, y_encoded, test_size=0.2, random_state=42)


In [10]:
rf_model_tfidf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model_tfidf.fit(X_train3, y_train3)

rf_pred3 = rf_model_tfidf.predict(X_test3)

print(classification_report(y_test3, rf_pred3, target_names=le.classes_))

y_pred_train_rf = rf_model_tfidf.predict(X_train3)
y_pred_test_rf = rf_model_tfidf.predict(X_test3)

accuracy_train_rf = accuracy_score(y_train3, y_pred_train_rf)
accuracy_test_rf = accuracy_score(y_test3, y_pred_test_rf)

print('Random Forest - accuracy train:', accuracy_train_rf)
print('Random Forest - accuracy test :', accuracy_test_rf)


              precision    recall  f1-score   support

    negative       0.78      0.94      0.85      5607
     neutral       0.68      0.25      0.37       635
    positive       0.86      0.68      0.76      3757

    accuracy                           0.80      9999
   macro avg       0.77      0.63      0.66      9999
weighted avg       0.80      0.80      0.79      9999

Random Forest - accuracy train: 0.9998999849977497
Random Forest - accuracy test : 0.8003800380038004


#Pelatihan: CNN, Ekstraksi Fitur: Embedding, Pembagian Data: 80/20

In [13]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X)

X_seq = tokenizer.texts_to_sequences(X)
maxlen = max(len(seq) for seq in X_seq)
X_pad = pad_sequences(X_seq, padding='post', maxlen=100)

ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_pad, y_encoded)

X_train, X_test, y_train, y_test = train_test_split(X_pad, y_encoded, test_size=0.2, random_state=42)

vocab_size = len(tokenizer.word_index) + 1

model_cnn = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=100),
    Conv1D(128, kernel_size=5, activation='relu'),
    GlobalMaxPooling1D(),
    Dropout(0.3),
    Dense(10, activation='relu'),
    Dropout(0.3),
    Dense(3, activation='softmax')
])

model_cnn.build(input_shape=(None, maxlen))
model_cnn.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 77, 64)         │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 73, 128)        │        41,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │         1,290 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 682,411 (2.60 MB)

 Trainable params: 682,411 (2.60 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
model_cnn.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history = model_cnn.fit(
              X_train, y_train,
              epochs=200,
              batch_size=32,
              validation_data=(X_test, y_test),
              callbacks=[early_stopping]
          )

cnn_pred = model_cnn.predict(X_test).argmax(axis=1)
print(classification_report(y_test, cnn_pred, target_names=le.classes_))

Epoch 1/200
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 37s 27ms/step - accuracy: 0.7122 - loss: 0.6940 - val_accuracy: 0.8602 - val_loss: 0.3861
Epoch 2/200
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 34s 27ms/step - accuracy: 0.8642 - loss: 0.3891 - val_accuracy: 0.8681 - val_loss: 0.3685
Epoch 3/200
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 41s 27ms/step - accuracy: 0.8906 - loss: 0.3030 - val_accuracy: 0.8714 - val_loss: 0.3765
Epoch 4/200
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 40s 26ms/step - accuracy: 0.9121 - loss: 0.2490 - val_accuracy: 0.8641 - val_loss: 0.3922
Epoch 5/200
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 34s 27ms/step - accuracy: 0.9257 - loss: 0.2070 - val_accuracy: 0.8573 - val_loss: 0.4801
Epoch 6/200
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 40s 27ms/step - accuracy: 0.9326 - loss: 0.1865 - val_accuracy: 0.8569 - val_loss: 0.4885
Epoch 7/200
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 41s 27ms/step - accuracy: 0.9352 - loss: 0.1694 - val_accuracy: 0.8599 - val_loss: 0.5398
Epoch 8/200
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 34s 27ms/step - accuracy: 

In [15]:
cnn_pred_train = model_cnn.predict(X_train).argmax(axis=1)
cnn_pred_test = model_cnn.predict(X_test).argmax(axis=1)

accuracy_train_cnn = accuracy_score(y_train, cnn_pred_train)
accuracy_test_cnn = accuracy_score(y_test, cnn_pred_test)

print("CNN - akurasi data train:", accuracy_train_cnn)
print("CNN - akurasi data test:", accuracy_test_cnn)

1250/1250 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step
CNN - akurasi data train: 0.9146121918287743
CNN - akurasi data test: 0.8680868086808681


#EKSPORT

In [16]:
import joblib
from google.colab import drive
drive.mount('/content/drive')

joblib.dump(svm_model, 'svm_model.pkl')
print("Model berhasil disimpan ke 'svm_model.pkl'")

joblib.dump(rf_model_tfidf, 'rf_model.pkl')
print("Model berhasil disimpan ke 'rf_model.pkl'")

Mounted at /content/drive
Model berhasil disimpan ke 'svm_model.pkl'
Model berhasil disimpan ke 'rf_model.pkl'


In [18]:
model_cnn.save('/content/drive/MyDrive/cnn_model.h5')
joblib.dump(svm_model, '/content/drive/MyDrive/svm_model.pkl')
joblib.dump(rf_model_tfidf, '/content/drive/MyDrive/rf_model_tfidf.pkl')

joblib.dump(tfidf, '/content/drive/MyDrive/tfidf_vectorizer.pkl')
joblib.dump(tokenizer, '/content/drive/MyDrive/tokenizer.pkl')

print("Model, tfidf, dan tokenizer berhasil diekspor ke Google Drive.")

Model, tfidf, dan tokenizer berhasil diekspor ke Google Drive.
